In [1]:
#YOLO 모델 불러오기
!git clone https://github.com/ultralytics/yolov5.git
%pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


Note: you may need to restart the kernel to use updated packages.


In [1]:
#데이터셋 다운로드
!kaggle datasets download -d a2015003713/militaryaircraftdetectiondataset

Dataset URL: https://www.kaggle.com/datasets/a2015003713/militaryaircraftdetectiondataset
License(s): unknown




  0%|          | 0.00/9.19G [00:00<?, ?B/s]
  0%|          | 1.00M/9.19G [00:00<48:32, 3.39MB/s]
  0%|          | 3.00M/9.19G [00:00<22:56, 7.17MB/s]
  0%|          | 4.00M/9.19G [00:00<20:51, 7.88MB/s]
  0%|          | 6.00M/9.19G [00:00<17:07, 9.59MB/s]
  0%|          | 8.00M/9.19G [00:00<15:47, 10.4MB/s]
  0%|          | 10.0M/9.19G [00:01<15:17, 10.7MB/s]
  0%|          | 12.0M/9.19G [00:01<14:48, 11.1MB/s]
  0%|          | 14.0M/9.19G [00:01<14:29, 11.3MB/s]
  0%|          | 16.0M/9.19G [00:01<14:31, 11.3MB/s]
  0%|          | 18.0M/9.19G [00:01<14:37, 11.2MB/s]
  0%|          | 20.0M/9.19G [00:02<14:24, 11.4MB/s]
  0%|          | 22.0M/9.19G [00:02<14:13, 11.5MB/s]
  0%|          | 24.0M/9.19G [00:02<14:07, 11.6MB/s]
  0%|          | 26.0M/9.19G [00:02<14:02, 11.7MB/s]
  0%|          | 28.0M/9.19G [00:02<14:05, 11.6MB/s]
  0%|          | 30.0M/9.19G [00:02<14:01, 11.7MB/s]
  0%|          | 32.0M/9.19G [00:03<13:59, 11.7MB/s]
  0%|          | 34.0M/9.19G [00:03<13:55, 11.8MB/s]
 

받은 zip 압축 풀어주세요

In [1]:
from pathlib import Path
import numpy as np
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10
np.random.seed(42)

import pandas as pd
import glob
import os

In [2]:
# 여러 파일에 접근 가능한 경로 설정
dataset_path = "militaryaircraftdetectiondataset\\dataset\\"
files = os.path.join(dataset_path, "*.csv")
files = glob.glob(files)

print("Joining all CSV files...")

# 파일 조인 시도
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
                               filename  width  height   class  xmin  ymin  \
0      0000e97ea2d086d6759b19b288a8a72c   4928    3264    Mi28  1380  1904   
1      0000e97ea2d086d6759b19b288a8a72c   4928    3264    Mi28  1809  1625   
2      0000e97ea2d086d6759b19b288a8a72c   4928    3264    Mi28  2400  1571   
3      0000e97ea2d086d6759b19b288a8a72c   4928    3264    Mi28  3935  1772   
4      00010041af654d0b8e1e16c824fa9867   1360    2048    UH60   835   526   
...                                 ...    ...     ...     ...   ...   ...   
31912  fff4d4af52e64d95bdafa29f77a81c97    817     479    Mi28     0   119   
31913  fffb908c9824dc0da884c43cfe0312b8   6720    4480  EF2000   514  1302   
31914  fffb908c9824dc0da884c43cfe0312b8   6720    4480  EF2000     0  1746   
31915  fffd70f851b78035454c0a9367820ca3   1500    1000      B2   240    55   
31916  ffff0595d9b782e9cd3c537529bf6027   1200     800    Su57   322    

In [3]:
# 데이터셋 클래스 판단
categories = list(set(df["class"]))
categories.sort()
print(categories)
print(len(categories))

['A10', 'A400M', 'AG600', 'AH64', 'AV8B', 'An124', 'An22', 'An225', 'An72', 'B1', 'B2', 'B21', 'B52', 'Be200', 'C130', 'C17', 'C2', 'C390', 'C5', 'CH47', 'CL415', 'E2', 'E7', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'H6', 'J10', 'J20', 'JAS39', 'JF17', 'JH7', 'KC135', 'KF21', 'KJ600', 'Ka27', 'Ka52', 'MQ9', 'Mi24', 'Mi26', 'Mi28', 'Mig29', 'Mig31', 'Mirage2000', 'P3', 'RQ4', 'Rafale', 'SR71', 'Su24', 'Su25', 'Su34', 'Su57', 'TB001', 'TB2', 'Tornado', 'Tu160', 'Tu22M', 'Tu95', 'U2', 'UH60', 'US2', 'V22', 'Vulcan', 'WZ7', 'XB70', 'Y20', 'YF23', 'Z19']
74


In [4]:
#학습(40%), 테스트(60%) 데이터 분리
train_data, val_data = train_test_split(files, test_size=0.4)
len(train_data), len(val_data)

(10965, 7311)

In [5]:
#YOLO 타입에 맞게 bounding box 데이터를 txt 파일로 변환환
def convert_csv_to_text(file, path):
    import csv
    csv_file = file
    txt_file = file.split('\\')[2][:-4] + '.txt'
    with open(os.path.join(path, txt_file), "w+") as my_output_file:
        with open(csv_file, "r") as my_input_file:
            reader = csv.reader(my_input_file)
            next(reader, None)
            for row in reader:
                category_idx = categories.index(row[3])
                xmin, ymin = int(row[4]), int(row[5])
                xmax, ymax = int(row[6]), int(row[7])
                bbox_width, bbox_height = int(row[1]), int(row[2])
                my_output_file.write(
                    f"{category_idx} {((xmin + xmax) / 2) / bbox_width} {((ymin + ymax) / 2) / bbox_height} {bbox_width/bbox_width} {bbox_height/bbox_height}\n"
                )
    my_input_file.close()
    my_output_file.close()
    return

In [6]:
# 폴더 생성성
!mkdir "{dataset_path}images\\train_data"
!mkdir "{dataset_path}labels\\train_data"
!mkdir "{dataset_path}images\\val_data"
!mkdir "{dataset_path}labels\\val_data"

In [7]:
import shutil
# 데이터셋 정리 및 변환환
for i in train_data:
    if i not in "militaryaircraftdetectiondataset\\dataset\\labels\\train_data":
        convert_csv_to_text(i, "militaryaircraftdetectiondataset\\dataset\\labels\\train_data")
        shutil.copy2(i[:-4] + ".jpg", "militaryaircraftdetectiondataset\\dataset\\images\\train_data")
for i in val_data:
    if i not in "militaryaircraftdetectiondataset\\dataset\\labels\\val_data":
        convert_csv_to_text(i, "militaryaircraftdetectiondataset\\dataset\\labels\\val_data")
        shutil.copy2(i[:-4] + ".jpg", "militaryaircraftdetectiondataset\\dataset\\images\\val_data")

In [8]:
# 해당 데이터셋에 맞는 yaml 생성성
f = open("detection.yaml", "w")
f.write("train: ../militaryaircraftdetectiondataset/dataset/images/train_data/")
f.write("\nval: ../militaryaircraftdetectiondataset/dataset/images/val_data/")
f.write("\nnc: " + str(len(categories)))
f.write("\nnames: [" + ', '.join("'" + str(i) + "'" for i in categories) + "]")
f.close()

# yaml 파일 읽기 시도
f = open("detection.yaml", "r")
print(f.read())
f.close()

train: ../militaryaircraftdetectiondataset/dataset/images/train_data/
val: ../militaryaircraftdetectiondataset/dataset/images/val_data/
nc: 74
names: ['A10', 'A400M', 'AG600', 'AH64', 'AV8B', 'An124', 'An22', 'An225', 'An72', 'B1', 'B2', 'B21', 'B52', 'Be200', 'C130', 'C17', 'C2', 'C390', 'C5', 'CH47', 'CL415', 'E2', 'E7', 'EF2000', 'F117', 'F14', 'F15', 'F16', 'F18', 'F22', 'F35', 'F4', 'H6', 'J10', 'J20', 'JAS39', 'JF17', 'JH7', 'KC135', 'KF21', 'KJ600', 'Ka27', 'Ka52', 'MQ9', 'Mi24', 'Mi26', 'Mi28', 'Mig29', 'Mig31', 'Mirage2000', 'P3', 'RQ4', 'Rafale', 'SR71', 'Su24', 'Su25', 'Su34', 'Su57', 'TB001', 'TB2', 'Tornado', 'Tu160', 'Tu22M', 'Tu95', 'U2', 'UH60', 'US2', 'V22', 'Vulcan', 'WZ7', 'XB70', 'Y20', 'YF23', 'Z19']


In [ ]:
# 학습시도도
import torch
torch.cuda.empty_cache()

#hyp.scratch-low.yaml에서 lr0을 0.001로 수정
!python yolov5/train.py --img 640 --batch 8 --epochs 20 --data detection.yaml --cfg yolov5/models/yolov5n.yaml --weights yolov5/runs/train/yolov5n_detection/weights/last.pt --name yolov5n_detection_N --cache disk --device 0 --hyp yolov5/data/hyps/hyp.scratch-low.yaml

In [3]:
#모델 테스트
!python yolov5/detect.py --weights yolov5/runs/train/yolov5n_detection/weights/best.pt --source militaryaircraftdetectiondataset/dataset/images/val_data/ --img 300 --conf 0.03 --iou 0.8 --device 0 --save-txt --save-conf --project yolov5/runs/test_results --name detection_test

detect: weights=['yolov5/runs/train/yolov5n_detection_N3/weights/best.pt'], source=militaryaircraftdetectiondataset/dataset/images/val_data/, data=yolov5\data\coco128.yaml, imgsz=[300, 300], conf_thres=0.03, iou_thres=0.8, max_det=1000, device=0, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/test_results, name=detection_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-388-g882c35fc Python-3.11.9 torch-2.3.0 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1859287 parameters, 0 gradients, 4.4 GFLOPs
WARNING  --img-size [300, 300] must be multiple of max stride 32, updating to [320, 320]
image 1/7311 E:\AirJet\model\militaryaircraftdetectiondataset\dataset\images\val_data\0000e97ea2d086d6759b19b288a8a72c.jpg: 224